In [ ]:
"""
textattack/bert-base-uncased-imdb: fine-tuned imdb using TextAttack, ~438 MB - https://huggingface.co/textattack/bert-base-uncased-imdb
"""
try:
    import torch, transformers, datasets
except:
    %pip install transformers datasets torch

In [1]:
import torch
from transformers import TextClassificationPipeline
from transformers import BertForSequenceClassification, BertTokenizer


review_texts = []
review_texts.append("I kinda liked it but ending was not what I expected.")
review_texts.append("I was blown away from the beginning to the end")
review_texts.append("The money well spent")
review_texts.append("I was sleeping like a baby")
review_texts.append("I am gonna come back to see the sequal")
review_texts.append("Do not waste money on the movie ticket")

# Load a pre-trained sentiment analysis model
model_id = "textattack/bert-base-uncased-imdb"  # ~438 MB
model = BertForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_id)


def get_movie_sentiment_pipeline(text):
    """
    Predict using transformer's pipeline method
    Input: Movie Review Text
    Output: ([Postive | Negative], probability)
    """

    pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)
    predictions = pipe(text)
    predicted_class = predictions[0]["label"]
    predicted_score = predictions[0]["score"]
    return predicted_class, predicted_score


def get_movie_sentiment_torch(text):
    """
    Predict using torch functions (instead of pipeline)
    Input: Movie Review Text
    Output: ([Postive | Negative], probability)
    """

    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs).logits

    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    predicted_class = torch.argmax(probabilities)
    predicted_score = torch.max(probabilities)
    return predicted_class, predicted_score


# Display sentiment result
for text in review_texts:
    sentiment, prob = get_movie_sentiment_pipeline(text)
    # sentiment, prob = get_movie_sentiment_torch(text)
    if sentiment == 1:
        sentiment = "Positive"
    else:
        sentiment = "Negative"
    print(f"{text} -> Sentiment: {sentiment} - {prob * 100:.2f}%")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


I kinda liked it but ending was not what I expected. -> Sentiment: Negative - 56.07%
I was blown away from the beginning to the end -> Sentiment: Negative - 99.46%
The money well spent -> Sentiment: Negative - 68.20%
I was sleeping like a baby -> Sentiment: Negative - 65.19%
I am gonna come back to see the sequal -> Sentiment: Negative - 64.05%
Do not waste money on the movie ticket -> Sentiment: Negative - 53.34%
